<a href="https://colab.research.google.com/github/bluebird702/study/blob/main/langchain_study.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

이 문서는 langchain study를 하면서 정리한 노트입니다.

# 실습 준비
colab에서 예제를 실행할 것이므로 Google AI Studio에 계정을 등록하고 Key를 얻는다. 자세한 내용은 다음 문서를 참고한다.

[Gemini Key 발급받아오기](https://colab.research.google.com/github/google/generative-ai-docs/blob/main/site/en/tutorials/python_quickstart.ipynb#scrollTo=gHYFrFPjSGNq)

# 환경 테스트
다음과 같이 테스트해보자. 공식 문서에 있는 내용을 그대로 테스트해본다.

[문서 링크](https://colab.research.google.com/github/google/generative-ai-docs/blob/main/site/en/tutorials/python_quickstart.ipynb#scrollTo=FFPBKLapSCkM)



In [ ]:
# google python sdk 설치
!pip install -q -U google-generativeai

In [ ]:
# 왼쪽의 열쇠 버튼에 GOOGLE_API_KEY를 등록한다음에 다음 코드를 돌려서 라이브러리를 초기화
from google.colab import userdata
import google.generativeai as genai

GOOGLE_API_KEY=userdata.get('GOOGLE_API_KEY')

genai.configure(api_key=GOOGLE_API_KEY)

for m in genai.list_models():
  if 'generateContent' in m.supported_generation_methods:
    display(m.name) # colab에서는 print 말고 display를 써야 wrapping된 결과를 얻을 수 있다.


'models/gemini-1.0-pro'

'models/gemini-1.0-pro-001'

'models/gemini-1.0-pro-latest'

'models/gemini-1.0-pro-vision-latest'

'models/gemini-1.5-pro-latest'

'models/gemini-pro'

'models/gemini-pro-vision'

In [ ]:
# 다음의 코드를 테스트해보자
import pathlib
import textwrap

from IPython.display import display
from IPython.display import Markdown

def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

In [ ]:
# gemini-pro를 일단 사용해본다.
model = genai.GenerativeModel('gemini-pro')

In [ ]:
# %%time은 맨 첫줄에서만 동작함. 그래서 코드셀을 나눴음.
%%time
response = model.generate_content("삶의 의미가 멀까?")
to_markdown(response.text)

In [ ]:
# Stream으로 노출하려면 출력을 다르게 해야한다.
%%time
response = model.generate_content("삶의 의미가 멀까?", stream=True)
for chunk in response:
  display(chunk.text)
  display("_"*80)

요정도에서 gemini설정을 마치고 langchain 학습으로 넘어간다.

langchain학습은 [테디노트님의 ebook](https://wikidocs.net/book/14314)으로 진행해본다.

# Ch01 LangChain 시작하기 노트
1장은 설정을 하고 있는데, openai로 예제가 나와 있다.

colab에서 테스트를 쉽게 하고 싶으니 gemini로 바꿔서 테스트해보자.

일단 langchain library를 설치한다.

In [ ]:
# LangChain 업데이트
!pip install -U langchain langchain-community langchain-experimental langchain-core langchain-google-genai langsmith

이제 잘되는 지 기본 예제 코드를 수행해본다.

In [ ]:
from langchain_google_genai import ChatGoogleGenerativeAI

# 객체 생성
llm = ChatGoogleGenerativeAI(
    model="gemini-pro",
    google_api_key=GOOGLE_API_KEY
)

# 물어보자.
result = llm.invoke("넌 누구냐!")
display(f"[답변]: {result.content}")

'[답변]: 저는 Gemini, Google에서 개발한 대규모 다국어 모델입니다.'

프롬프트 템플릿도 한번 써보자.

일단 prompt를 만든다.

In [ ]:
from langchain.prompts import PromptTemplate

template = "{country}의 수도는 뭐야?"

# 템플릿 완성
prompt = PromptTemplate.from_template(template=template)
prompt

PromptTemplate(input_variables=['country'], template='{country}의 수도는 뭐야?')

그 다음에 prompt를 써서 요청을 날려보자

In [ ]:
from langchain.chains import LLMChain

llm_chain = LLMChain(prompt=prompt, llm=llm) # 위에서 prompt와 llm을 만드는 코드를 실행한 후에 여기를 돌려야 한다.

llm_chain.invoke({"country": "대한민국"}) # 요거는 서울

{'country': '대한민국', 'text': '서울'}

apply()로 여러개의 입력도 한번에 처리가 가능하다.

In [ ]:
input_list = [{"country": "호주"}, {"country": "중국"}, {"country":"네덜란드"}]

result = llm_chain.apply(input_list)

display(result)

for res in result:
  display(res["text"].strip())

[{'text': '캔버라'}, {'text': '베이징'}, {'text': '암스테르담'}]

'캔버라'

'베이징'

'암스테르담'

generate()로 좀 더 자세한 추가정보를 출력할 수 있다.

In [ ]:
input_list = [{"country": "호주"}, {"country": "중국"}, {"country":"네덜란드"}]

result = llm_chain.generate(input_list)

display(result)

토큰 사용량을 바로 보려면 다음과 같이 찍으면 된다고 하는데 OpenAI는 되나본데, Gemini는 아무것도 안나온다. 따로 방법을 찾아보자.

In [ ]:
result.llm_output

{}

2개 이상의 변수를 템플릿 안에 정의하는 것도 가능하다.

In [ ]:
template = "{area1}와 {area2}의 시차는 몇시간이야?"

prompt = PromptTemplate.from_template(template)
prompt

In [ ]:
llm_chain = LLMChain(prompt=prompt, llm=llm)

In [ ]:
display(llm_chain.invoke({"area1": "서울", "area2": "파리"}))

In [ ]:
input_list = [
    {"area1": "파리", "area2": "뉴욕"},
    {"area1": "서울", "area2": "하와이"},
    {"area1": "켄버라", "area2": "베이징"},
]

# 반복문으로 결과 출력
result = llm_chain.apply(input_list)
for res in result:
    display(res["text"].strip())

stream으로 출력도 가능하다만 OpenAI로 좀 다른거 같다. stream이란 함수를 호출해서 출력한다.

In [ ]:
llm = ChatGoogleGenerativeAI(
    model="gemini-pro",
    google_api_key=GOOGLE_API_KEY,
    temperature=0, #창의성
    max_output_tokens=2048,
)

question = "대한민국에 대해서 300자 내외로 최대한 상세히 알려줘"

for chunk in llm.stream(question):
    display(to_markdown(chunk.content))
    display(to_markdown("---"))

Chain 생성에서 LCEL(LangChain Expression Language)도 가능한지 살펴본다.

In [ ]:
from langchain_core.output_parsers import StrOutputParser

# 주어진 나라에 대하여 수도를 묻는 프롬프트 템플릿을 생성합니다.
template = """
당신은 친절하게 답변해 주는 친절 봇입니다. 사용자의 질문에 [FORMAT]에 맞추어 답변해 주세요.
답변은 항상 한글로 작성해 주세요.

질문:
{question}에 대하여 설명해 주세요.

FORMAT:
- 개요:
- 예시:
- 출처:
"""

template = """
당신은 영어를 가르치는 10년차 영어 선생님입니다. 상황에 [FORMAT]에 영어 회화를 작성해 주세요.

상황:
{question}

FORMAT:
- 영어 회화:
- 한글 해석:
"""

prompt = PromptTemplate.from_template(template)

# OpenAI 챗모델을 초기화합니다.
model = ChatGoogleGenerativeAI(
    model="gemini-pro",
    google_api_key=GOOGLE_API_KEY,
    temperature=0, #창의성
    max_output_tokens=2048,
)

# 문자열 출력 파서를 초기화합니다.
output_parser = StrOutputParser()

# 프롬프트, 모델, 출력 파서를 연결하여 처리 체인을 구성합니다.
chain = prompt | model | output_parser

# 완성된 Chain 을 이용하여 country 를 '대한민국'으로 설정하여 실행합니다.
# chain.invoke({"country": "대한민국"})
print(chain.invoke({"question": "저는 식당에 가서 음식을 주문하고 싶어요"}))

In [ ]:
print(chain.invoke({"question": "미국에서 피자 주문"}))

ebook에 있는 다른 예제도 실행해본다.

In [ ]:
# prompt 를 PromptTemplate 객체로 생성합니다.
prompt = PromptTemplate.from_template("{topic} 에 대해 쉽게 설명해주세요.")

# input 딕셔너리에 주제를 'ice cream'으로 설정합니다.
input = {"topic": "양자역학"}

# prompt 객체의 invoke 메서드를 사용하여 input을 전달하고 대화형 프롬프트 값을 생성합니다.
prompt.invoke(input)

# prompt 객체와 model 객체를 파이프(|) 연산자로 연결하고 invoke 메서드를 사용하여 input을 전달합니다.
# 이를 통해 AI 모델이 생성한 메시지를 반환합니다.
(prompt | model).invoke(input)

In [ ]:
# parse_output 메서드를 사용하여 AI 모델이 생성한 메시지 문자열로 출력합니다.
display((prompt | model | output_parser).invoke(input))

LCEL 인터페이스에 있는 다른 내용은 필요할 때 보면 될 것 같고, 병렬성 예제를 돌려본다.

In [ ]:
from langchain.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableParallel

# {country} 의 수도를 물어보는 체인을 생성합니다.
chain1 = ChatPromptTemplate.from_template("{country} 의 수도는 어디야?") | model

# {country} 의 면적을 물어보는 체인을 생성합니다.
chain2 = ChatPromptTemplate.from_template("{country} 의 면적은 얼마야?") | model
# 위의 2개 체인을 동시에 생성하는 병렬 실행 체인을 생성합니다.
combined = RunnableParallel(capital=chain1, area=chain2)

In [ ]:
chain1.invoke(
    {"country": "대한민국"}
)

In [ ]:
chain2.invoke({"country": "미국"})

In [ ]:
# 주어진 'country'에 대해 'combined' 객체의 'invoke' 메서드를 호출합니다.
combined.invoke({"country": "대한민국"})